In [1]:
from bottle import Bottle, route, run, template, get, post, debug, static_file, request, redirect, response
import time
import random
import string
import logging
import logging.handlers
import sqlite3

log = logging.getLogger('bottle')
log.setLevel('INFO')
h = logging.handlers.TimedRotatingFileHandler(
    'logs/nlog', when='midnight', backupCount=9999)
f = logging.Formatter('%(asctime)s %(levelname)-8s %(message)s')
h.setFormatter(f)
log.addHandler(h)

app = Bottle()

@app.route('/static/:path#.+#', name='static')
def static(path):
    return static_file(path, root='./static')

@app.route('/')
@app.route('/index/')
def index():
    return template('index')

@app.route('/register_page')
def register1():
    return template('register1')

def CheckIfCorrectCredentials(loginName,password):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params=[loginName,password]
    c.execute("Select * from users where loginName==? and password==?",params)
    print("\nPresent all data in student table:")

    #Get all results,assign them to the list,fecthall() returns empty list if no results
    listOfResults=c.fetchall()

    # Save (commit) the changes
    conn.commit()

    # We close the connection and free all resources
    conn.close()
    
    # If given loginName and password exist in database then exactly one row should be returned.
    # We assume that loginNames are unique.
    if (len(listOfResults)==1):
        return True
    return False

@app.route('/login_page')
def login_page():
    return template('login')

@app.route('/login/')
@app.route('/login', method='POST')
def login():
    loginName = request.forms.get('login_name', default=False)
    password = request.forms.get('password', default=False)

    log.info(str(loginName) + ' ' + request.method + ' ' +
             request.url + ' ' + request.environ.get('REMOTE_ADDR'))

    if (CheckIfCorrectCredentials(loginName,password)==True):  
        redirect('/main')
        return True
    else:
        return template('login')
    return template('login')

@app.route('/main')
def main():
    return template('main')


def RegisterUserPersonalInfo(name, surname, birthDate):

    conn = sqlite3.connect('DataBase1.db')

    c = conn.cursor()

    params = [name, surname, birthDate]
    c.execute("insert into userPersonalInfo values (2, ?, ?, ?, 1)", params)
    conn.commit()
    conn.close()

@app.route('/register')
@app.route('/register', method="POST")
def register():
    loginName = request.forms.get('login_name', default=False)
    password1 = request.forms.get('password1', default=False)
    password2 = request.forms.get('password2', default=False)
    name = request.forms.get('name', default=False)
    surname = request.forms.get('surname', default=False)
    birthDate = request.forms.get('birthDate', default=False)
    city = request.forms.get('city', default=False)
    zipCode = request.forms.get('zipCode', default=False)
    street = request.forms.get('street', default=False)
    houseNo = request.forms.get('houseNo', default=False)
    flatNo =request.forms.get('flatNo', default=False)

    RegisterUserPersonalInfo(name, surname, birthDate)

    return template('main')



app.run(host='localhost', port=63700, reloader=False, debug=False)

Bottle v0.12.23 server starting up (using WSGIRefServer())...
Listening on http://localhost:63700/
Hit Ctrl-C to quit.

127.0.0.1 - - [04/Jan/2023 23:25:50] "POST /register HTTP/1.1" 200 224
